In [ ]:
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install bitsandbytes transformers accelerate peft -q
pip install --upgrade transformers
!pip install git+https://github.com/huggingface/diffusers.git -q
pip install --upgrade torch torchvision torchaudio accelerate
# Update all related packages to ensure compatibility
!pip install --upgrade diffusers transformers accelerate peft

In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --instance_data_dir="/content/drive/MyDrive/dragons/images" \
  --output_dir="/content/dragons_lora" \
  --mixed_precision="fp16" \
  --instance_prompt="a detailed YU-GI-Oh dragon monster with intricate artwork" \
  --resolution=512 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --snr_gamma=5.0 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_8bit_adam \
  --max_train_steps=400 \
  --checkpointing_steps=50 \
  --seed=0

In [ ]:
# COLAB OPTIMIZED: Bulk YU-GI-OH Dragon Generation (1000 Images)
# Run this cell after your LoRA training is complete

import torch
from diffusers import StableDiffusionXLPipeline
import os
import zipfile
import random
from tqdm.notebook import tqdm  # Colab-optimized progress bar
import gc
import time
from PIL import Image
import matplotlib.pyplot as plt

print("🐉 COLAB YU-GI-OH DRAGON BULK GENERATOR")
print("=" * 50)

# Check GPU and memory
print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create output directory
output_dir = "/content/dragon_collection_2"
os.makedirs(output_dir, exist_ok=True)
print(f"📁 Output directory: {output_dir}")

# COLAB-OPTIMIZED PIPELINE LOADING
print("\n🚀 Loading pipeline...")

# Check if pipeline already exists in memory (from training)
if 'pipe' not in globals():
    print("Loading fresh pipeline...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Colab memory optimizations
    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()
    pipe = pipe.to("cuda")

    # Load LoRA
    pipe.load_lora_weights("/content/dragons_lora")
    print("✅ Fresh pipeline loaded with LoRA")
else:
    print("✅ Using existing pipeline from training")

# YU-GI-OH DRAGON PROMPTS (Colab optimized list)
dragon_prompts = [
    "a YU-GI-OH Blue-Eyes White Dragon, majestic pose, trading card art",
    "a YU-GI-OH Red-Eyes Black Dragon, fierce expression, dark flames",
    "a YU-GI-OH Cyber Dragon, metallic silver body, mechanical details",
    "a YU-GI-OH Thunder Dragon, electric energy, storm clouds",
    "a YU-GI-OH Crystal Dragon, transparent crystalline body, prismatic light",
    "a YU-GI-OH Shadow Dragon, dark aura, mysterious atmosphere",
    "a YU-GI-OH Fire Dragon, blazing wings, volcanic background",
    "a YU-GI-OH Ice Dragon, frozen breath, winter landscape",
    "a YU-GI-OH Earth Dragon, stone armor, mountain setting",
    "a YU-GI-OH Wind Dragon, tornado effects, sky background",
    "a YU-GI-OH Light Dragon, golden radiance, celestial aura",
    "a YU-GI-OH Dark Dragon, purple energy, nightmare realm",
    "a YU-GI-OH Machine Dragon, robotic features, cyber world",
    "a YU-GI-OH Ancient Dragon, mystical symbols, temple ruins",
    "a YU-GI-OH Elemental Dragon, multi-colored energy, magic circles"
]

styles = [
    "official YU-GI-OH card artwork style",
    "epic and detailed creature design",

]

def get_random_prompt():
    """Generate random YU-GI-OH dragon prompt"""
    base = random.choice(dragon_prompts)
    style = random.choice(styles)
    return f"{base}, {style}"

# COLAB-OPTIMIZED GENERATION FUNCTION
def generate_dragons_colab(total_count=200, batch_size=10):
    """
    Generate dragons optimized for Colab environment
    """

    print(f"\n🎨 Generating {total_count} YU-GI-OH dragon images")
    print(f"⚡ Batch size: {batch_size} (optimized for Colab)")
    print(f"⏱️ Estimated time: {total_count * 1.5 / 60:.0f}-{total_count * 2.5 / 60:.0f} minutes")

    # Colab progress bar
    progress_bar = tqdm(total=total_count, desc="🐉 Dragons Generated")

    generated_count = 0
    batch_count = 0

    counter = 0

    try:
        for i in range(0, total_count, batch_size):
            batch_count += 1
            current_batch_size = min(batch_size, total_count - i)

            print(f"\n🔄 Batch {batch_count} ({i+1}-{i+current_batch_size})")

            for j in range(current_batch_size):
                img_num = i + j + 1

                try:
                    # Generate random prompt and seed
                    prompt = get_random_prompt()
                    seed = random.randint(1, 999999)

                    print(prompt)

                    # Generate image
                    generator = torch.Generator(device="cuda").manual_seed(seed)
                    result = pipe(
                        prompt=prompt,
                        negative_prompt="blurry, low quality, distorted, deformed, ugly",
                        num_inference_steps=25,  # Balanced speed/quality for bulk generation
                        guidance_scale=7.5,
                        width=1024,
                        height=1024,
                        generator=generator
                    )

                    # Save image
                    filename = f"{file_names[counter]}.png"
                    filepath = os.path.join(output_dir, filename)
                    result.images[0].save(filepath)

                    generated_count += 1
                    counter += 1
                    progress_bar.update(1)

                except Exception as e:
                    print(f"❌ Error on image {img_num}: {str(e)}")
                    continue

            # Colab memory management every batch
            if batch_count % 3 == 0:  # Every 3 batches
                torch.cuda.empty_cache()
                gc.collect()
                print(f"🧹 Memory cleanup completed")

    except KeyboardInterrupt:
        print(f"\n⛔ Generation stopped by user at {generated_count} images")

    progress_bar.close()
    return generated_count

# COLAB PREVIEW FUNCTION
def show_preview_grid(num_preview=9):
    """Show a 3x3 grid of generated images in Colab"""

    files = [f for f in os.listdir(output_dir) if f.endswith('.png')]

    if len(files) == 0:
        print("No images found to preview")
        return

    # Select random images for preview
    preview_files = random.sample(files, min(num_preview, len(files)))

    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    fig.suptitle("🐉 Generated YU-GI-OH Dragons Preview", fontsize=16)

    for i, filename in enumerate(preview_files):
        row, col = i // 3, i % 3

        img_path = os.path.join(output_dir, filename)
        img = Image.open(img_path)

        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        axes[row, col].set_title(f"Dragon #{filename[13:17]}", fontsize=10)

    # Hide empty subplots
    for i in range(len(preview_files), 9):
        row, col = i // 3, i % 3
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()

# COLAB DOWNLOAD FUNCTION
def create_and_download_zip():
    """Create zip and download in Colab"""

    files = [f for f in os.listdir(output_dir) if f.endswith('.png')]

    if len(files) == 0:
        print("❌ No images to zip!")
        return

    zip_path = "/content/yugioh_dragons_collection.zip"

    print(f"📦 Creating zip with {len(files)} images...")

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
        for filename in tqdm(files, desc="Zipping"):
            file_path = os.path.join(output_dir, filename)
            zipf.write(file_path, filename)

    # Get zip size
    zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"✅ Zip created: {zip_size_mb:.1f} MB")

    # Colab download
    from google.colab import files
    print("⬇️ Starting download...")
    files.download(zip_path)

    return zip_path

# MAIN EXECUTION FOR COLAB
print("\n" + "="*50)
print("🚀 READY TO GENERATE!")
print("Run the cells below to start generation:")

print("\n📋 STEP 1: Generate Images")
print("generated_count = generate_dragons_colab(1000, batch_size=10)")

print("\n📋 STEP 2: Preview Results")
print("show_preview_grid()")

print("\n📋 STEP 3: Download Collection")
print("create_and_download_zip()")

print("\n" + "="*50)

# AUTO-RUN OPTION (uncomment to run automatically)
print("🔥 AUTO-GENERATION STARTING IN 5 SECONDS...")
print("Press Ctrl+C to cancel")

try:
    time.sleep(5)

    # Start automatic generation
    print("\n🎬 STARTING AUTOMATIC GENERATION")
    generated_count = generate_dragons_colab(1000, batch_siz=10)

    print(f"\n🎉 GENERATION COMPLETE!")
    print(f"📊 Successfully generated: {generated_count} images")

    # Show preview
    print("\n🖼️ Showing preview...")
    show_preview_grid()

    # Create and download zip
    print("\n📦 Creating download package...")
    create_and_download_zip()

    print("\n✨ ALL DONE! Your dragon collection is downloading!")

except KeyboardInterrupt:
    print("\n⛔ Auto-generation cancelled")
    print("You can run the functions manually:")
    print("- generate_dragons_colab(1000)")
    print("- show_preview_grid()")
    print("- create_and_download_zip()")